In [ ]:
%matplotlib inline
import os
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyemu
import flopy

In [ ]:
import freyberg_setup as fs
pst_name = fs.PST_NAME_ZN
working_dir = fs.WORKING_DIR_ZN
fs.setup_pest_zn()

In [ ]:
fs.plot_model(working_dir, pst_name)

## Given a zone array let's plot it up

In [ ]:
zones = np.loadtxt(os.path.join(working_dir, 'hk.zones'))
plt.imshow(zones)
plt.colorbar()

In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))


In [ ]:
pst.parameter_data

## run pestpp

In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.control_data.noptmax = 10
pst.write(os.path.join(working_dir,pst_name))

In [ ]:
os.chdir(working_dir)
pyemu.helpers.start_slaves('.',"pestpp",pst_name,num_slaves=12,master_dir='.')
os.chdir("..")

In [ ]:
pst.phi

In [ ]:
pst.phi_components

In [ ]:
pst.res.loc[pst.nnz_obs_names,:]

In [ ]:
pyemu.plot_utils.res_1to1(pst)

In [ ]:
df_paru = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".par.usum.csv")),index_col=0)
df_paru

In [ ]:
axes = pyemu.helpers.plot_summary_distributions(df_paru,subplots=True)

## Why are ``rch_1`` and `porosity` unaffected by calibration? And why did ``hk_z01`` change so little?

## We can look at the sensitivity

In [ ]:
sens = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".isen")))
sens

## What's up with ``hk_z01``?

In [ ]:
# look at ibound
ib = np.loadtxt(os.path.join(working_dir,'ibound_layer_1.ref'))
zones = np.loadtxt(os.path.join(working_dir, 'hk.zones'))


In [ ]:
fig = plt.figure(figsize=(8,4))
fig.add_subplot(131)
plt.imshow(ib)
plt.colorbar()
plt.title('ibound')
fig.add_subplot(132)
plt.imshow(zones)
plt.colorbar()
plt.title('zones')
fig.add_subplot(133)
zones[ib!=1]=np.nan
plt.imshow(zones)
plt.title('zones censored')
plt.colorbar()
plt.tight_layout()


In [ ]:
df_predu = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".pred.usum.csv")),index_col=0)

In [ ]:
figs, axes = pyemu.helpers.plot_summary_distributions(df_predu,subplots=True)
for ax in axes:
    fname = ax.get_title().lower()
    ylim = ax.get_ylim()
    v = pst.observation_data.loc[fname,"obsval"]
    ax.plot([v,v],ylim,"k--")
    ax.set_ylim(ylim)

## Now we have some variability in the forecast outcomes...why?

## As far as the "truth" is concerned, we are doing better for most forecasts.  But we are especially bad for the travel time forecast - why?  What affects travel time?

## Zone Burner approach

In [ ]:
zones = np.loadtxt(os.path.join(working_dir, 'hk.zones'))

np.unique(zones)

In [ ]:

a = zones.copy()
newk = [100,222,303,444,515,676]

In [ ]:
for cz in np.unique(zones):
    print('updating zone {}'.format(cz))
    a[zones==cz] = newk[int(cz)-1]

In [ ]:
a

In [ ]:
plt.imshow(a)
plt.colorbar()